# Выпускной проект. Оценка результатов A/B-теста.

**Структура проекта:**

1. Вводная часть
    - Цель исследования
    - Техническое задание 
    - Описание данных
---
2. Исследование данных/Предобработка
    - Загрузка данных/изучение
    - Преобразование типов данных
    - Пропуски дубликаты
---
3. Оценка корректности проведения теста
    - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
    - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
    - Аудитория теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения пользователей по тестовым группам и правильность их формирования.
---
4. Проведите исследовательский анализ данных
    - Количество событий на пользователя одинаково распределены в выборках?
    - Как число событий в выборках распределено по дням?
    - Как меняется конверсия в воронке в выборках на разных этапах?
    - Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
---
5. Проведите оценку результатов A/B-тестирования:
    - Что можно сказать про результаты A/B-тестирования?
    - Проверьте статистическую разницу долей z-критерием.
---
6. Общий вывод

## Вводная часть

**Цель исследования:**

Оценить корректность проведения теста и проанализировать его результаты.  

Чтобы оценить корректность проведения теста:
- *необходимо удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно;*
- *необходимо проверить равномерность распределения пользователей по тестовым группам и правильность их формирования.*

В арсенале есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.



**Техническое задание:**
- Название теста: `recommender_system_test`;

- Группы: `А (контрольная), B (новая платёжная воронка)`;

- Дата запуска: `2020-12-07`;

- Дата остановки набора новых пользователей: `2020-12-21`;

- Дата остановки: `2021-01-04`;

- Аудитория: `15% новых пользователей из региона EU`;

- Назначение теста: `тестирование изменений, связанных с внедрением улучшенной рекомендательной системы`;

- Ожидаемое количество участников теста: `6000`.

- Ожидаемый эффект: `за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%`;

    - Kонверсии в просмотр карточек товаров — `событие product_page`

    - просмотры корзины — `product_cart`

    - покупки — `purchase`.


**Описание данных:**

`/datasets/ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год;

Структура файла:
- name — название маркетингового события;
- regions — регионы, в которых будет проводиться рекламная кампания;
- start_dt — дата начала кампании;
- finish_dt — дата завершения кампании.

---

`/datasets/final_ab_new_users.csv` — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года;

Структура файла:
- user_id — идентификатор пользователя;
- first_date — дата регистрации;
- region — регион пользователя;
- device — устройство, с которого происходила регистрация.

---

`/datasets/final_ab_events.csv` — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;

Структура файла:
- user_id — идентификатор пользователя;
- event_dt — дата и время события;
- event_name — тип события;
- details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

---

`/datasets/final_ab_participants.csv` — таблица участников тестов.

Структура файла:
- user_id — идентификатор пользователя;
- ab_test — название теста;
- group — группа пользователя.

## Исследование данных/Предобработка

### Загрузка данных/изучение

In [37]:
#Загрузим все необходимые библиотеки
import pandas as pd
import numpy as np
import datetime
from datetime import date, datetime, timedelta

import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from statsmodels.stats.weightstats import ztest as ztest

In [38]:
#Сохраним таблицы в переменные
marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')

In [39]:
#Изучение общей информации
def research_source(table):
    display(table.head(), display(table.info()))

scours = [marketing_events, new_users, events, participants]
    
for x in scours:
    print(research_source(x))
    print('#'*100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


None

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


None

None
####################################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


...

None
####################################################################################################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


None

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


None

None
####################################################################################################


### Преобразование типов данных

`marketing_events` - календарь маркетинговых событий на 2020 год

In [40]:
#Преобразуем start_dt и finish_dt в формат дат
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])

---

`new_users` - все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года

In [41]:
#Преобразуем first_date в формат дат
new_users['first_date'] = pd.to_datetime(new_users['first_date'])

---

`events` - все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года

In [42]:
#Преобразуем first_date в формат дат
events['event_dt'] = pd.to_datetime(events['event_dt'])

---

### Пропуски дубликаты

In [43]:
#Напишем функцию на проверку дублей и пропусков
def dubl_miss(df):
    print(('В датассете {} дубликатов').format(df.duplicated().sum()))
    print('')
    print(('В датассете представлены пропуски'))
    print(df.isna().sum())

scours = ['marketing_events', 'new_users', 'events', 'participants']
scours_2 = [marketing_events, new_users, events, participants]

for x, y in zip(scours, scours_2):
    print('Название датассета', x)
    print(dubl_miss(y))
    print('#'*50)


Название датассета marketing_events
В датассете 0 дубликатов

В датассете представлены пропуски
name         0
regions      0
start_dt     0
finish_dt    0
dtype: int64
None
##################################################
Название датассета new_users
В датассете 0 дубликатов

В датассете представлены пропуски
user_id       0
first_date    0
region        0
device        0
dtype: int64
None
##################################################
Название датассета events
В датассете 0 дубликатов

В датассете представлены пропуски
user_id            0
event_dt           0
event_name         0
details       377577
dtype: int64
None
##################################################
Название датассета participants
В датассете 0 дубликатов

В датассете представлены пропуски
user_id    0
group      0
ab_test    0
dtype: int64
None
##################################################


In [44]:
#Проверим природу пропусков в датассете final_ab_events найдем колличество строк с типом собития purchase
events[events['event_name'] == 'purchase'].count()

user_id       62740
event_dt      62740
event_name    62740
details       62740
dtype: int64

In [45]:
#Найдем количество не пустых срок в стоблце с пропусками 
events[events['details'] != events['details'].isna()].count()

user_id       440317
event_dt      440317
event_name    440317
details        62740
dtype: int64

Количество строк соответсвует, а значит мы можем сделать вывод, что наличие заполненых строк в `details` обосновано тем, что там помечается сумма покупки в долларах.

**Вывод:**

- Загрузка данных/изучение, мы подгрузили 4 датассета и ознакомились с ними (просмотрели типы и природу самих таблиц)

- Преобразование типов данных, в 3 из 4 датассетов мы заменили типы столбцов с датой, на datetime.

- Пропуски и дубликаты, посмотрели природу пропусков и дубликатов, нашли обоснование пропускам в таблице final_ab_events.

## Оценка корректности проведения теста

### Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.

In [46]:
#Посмотрим на уникальные значения столбца с названием теста
participants.groupby(['ab_test','group']).agg(count = ('user_id','count'))

count
ab_test                 group       
interface_eu_test       A       5831
                        B       5736
recommender_system_test A       3824
                        B       2877

Представлено два теста: `recommender_system_test` и `interface_eu_test`, нужно узнать, сколько пользователей вошло в оба теста и убрать их.

In [47]:
#Найдем количество пользователей, которые участвовали в двух тестах сразу
tests = participants.groupby('user_id').agg({'ab_test': 'nunique'}).sort_values(by = 'ab_test', ascending = False).reset_index()
print('Количество пользователей принимавших участие в двух тестах:', tests[tests['ab_test'] == 2]['user_id'].nunique())

#выведем отдельно этот список пользователей 
users_intwo_test = tests[tests['ab_test'] == 2]['user_id']

Количество пользователей принимавших участие в двух тестах: 1602


In [48]:
#По ТЗ необходимый нам тест recommender_system_test - оставим его
participants = participants[participants['ab_test'] == 'recommender_system_test']

In [49]:
#Дата запуска: 2020-12-07
new_users['first_date'].min()

Timestamp('2020-12-07 00:00:00')

Ничего менять не надо

In [50]:
#Дата остановки набора новых пользователей: 2020-12-21
new_usersss = new_users[new_users['first_date'] <= '2020-12-21']
new_usersss['first_date'].max()

Timestamp('2020-12-21 00:00:00')

Урезали дату остановки набора. Итого получаем дата запуска теста: `2020-12-07`, а дата остановки набора пользователей `2020-12-21`.

In [51]:
# сделаем фильтр по нужному нам тесту
fun_test = new_usersss[new_usersss['user_id'].isin(participants[participants['ab_test'] == 'recommender_system_test']['user_id'])]

In [52]:
#смотрим дату последнего события
events['event_dt'].max()

Timestamp('2020-12-30 23:36:33')

*Дата последнего события - 30 декабря 2020, по ТЗ тест должен длится до 4 января 2021.*

In [53]:
#Посмотрим % аудитории из региона EU
fun_test[fun_test['region'] == 'EU']['user_id'].count()/new_usersss[new_usersss['region'] == 'EU']['user_id'].count()*100

15.0

In [54]:
#количество участников по ТЗ 6000
participants['user_id'].nunique()

6701

**Вывод**

По итогам проработки пункта "Оценка корректности проведения теста" отметим:
- Всего проводилось два теста `interface_eu_test` и `recommender_system_test` 
- Количсетво пользователей, которые участвовали в двух тестах равно 1602
- Группа А — контрольная, B — новая платёжная воронка
- Даты запуска теста - 2021.01.04 и дата окончания набора новых пользователей 2020.12.21 верны
- Аудитория теста у новых пользователей с Европы равна 15%, что соответствует ТЗ
- Ожидаемое количество участников теста: 6000. У нас количество участиков 6701.

### Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.

In [55]:
#построим таблицу
marketing_events.query('finish_dt >= "2020-12-07" and start_dt <= "2021-01-04"')

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
10,CIS New Year Gift Lottery,CIS,2020-12-30,2021-01-07


**Вывод:**

В временной интервал проведения теста попали две акции `Christmas&New Year Promo` и `CIS New Year Gift Lottery`. Но первая акция окажет большее влиияние, нежели чем вторая, потому что данная акция проводилась именно в Европе.

### Аудитория теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения пользователей по тестовым группам и правильность их формирования.

Ранее мы выяснили, что 1602 пользователя участвуют в двух тестах одновременно.

In [56]:
#проверим присутствуют ли пользователи которыe, входящят в обе группы А и В теста recommender_system_test
a = participants[participants['group'] == 'A']
b = participants[participants['group'] == 'B']

a[a['user_id'].isin(b['user_id'])]

,user_id,group,ab_test


Пользователей не обнаружено

In [57]:
group = participants.groupby('group').agg(
    user_id_cnt = ('user_id','nunique'),
    cnt_=('user_id', lambda x: "{0:.0%}".format(x.count() / participants['user_id'].count()))
)
group

,user_id_cnt,cnt_
group,,
A,3824,57%
B,2877,43%


В группе А `3824` пользователя - 57% от общего количества, 
а в группе В `2877` - 43% от общего количества. Пользователи в группах распределены неравномерно.

In [58]:
#объединим все таблицы (по необходимому тесту)
fin = fun_test.merge(participants.query('ab_test == "recommender_system_test"'), on='user_id')
fin = fin.merge(events, how='left', on='user_id')
fin['user_id'].nunique()

6701

In [59]:
#Подготовим фильтр в 14 дней

horizon_days = 14 #горизонт
observation_date = datetime(2021, 1, 4).date() #последняя дата наблюдения
last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days - 1) #последняя дата анализа


fourteen_days = fin.query('event_dt <= @last_suitable_acquisition_date')

#Лайфтайм
fourteen_days['lifetime'] = (fourteen_days['event_dt'] - fourteen_days['first_date']).dt.days
fourteen_days = fourteen_days.query('lifetime < 14')
fourteen_days['user_id'].nunique()

/tmp/ipykernel_48/4199740187.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3675

## Исследовательский анализ данных

### Количество событий на пользователя одинаково распределены в выборках?

In [60]:
fourteen_days['event_name'].unique()
fourteen_days

,user_id,first_date,region,device,group,ab_test,event_dt,event_name,details,lifetime
0,D72A72121175D8BE,2020-12-07,EU,PC,A,recommender_system_test,2020-12-07 21:52:10,product_page,NaN,0
1,D72A72121175D8BE,2020-12-07,EU,PC,A,recommender_system_test,2020-12-07 21:52:07,login,NaN,0
3,DD4352CDCF8C3D57,2020-12-07,EU,Android,B,recommender_system_test,2020-12-07 15:32:54,product_page,NaN,0
4,DD4352CDCF8C3D57,2020-12-07,EU,Android,B,recommender_system_test,2020-12-08 08:29:31,product_page,NaN,1
5,DD4352CDCF8C3D57,2020-12-07,EU,Android,B,recommender_system_test,2020-12-10 18:18:27,product_page,NaN,3
...,...,...,...,...,...,...,...,...,...,...
27715,0416B34D35C8C8B8,2020-12-20,EU,Android,A,recommender_system_test,2020-12-21 22:28:29,purchase,4.99,1
27717,0416B34D35C8C8B8,2020-12-20,EU,Android,A,recommender_system_test,2020-12-20 20:58:26,product_page,NaN,0
27718,0416B34D35C8C8B8,2020-12-20,EU,Android,A,recommender_system_test,2020-12-21 22:28:29,product_page,NaN,1
27720,0416B34D35C8C8B8,2020-12-20,EU,Android,A,recommender_system_test,2020-12-20 20:58:25,login,NaN,0


In [61]:
#количество событий по пользователями и событиям  
count_events_on_users_and_events = fourteen_days.groupby(['event_name', 'user_id'], as_index=False).agg({'device': 'count'})\
             .rename(columns={'device':'events_count'})\
             .merge(fourteen_days[['user_id', 'group']], on='user_id')\
             .drop_duplicates()

#количество событий по группам
count_events_on_users_and_events_group = count_events_on_users_and_events.groupby(['group', 'event_name'], as_index=False)['events_count'].agg('sum')
count_events_on_users_and_events_group 

#количество оставшихся в тесте пользователей в группах А и В
A = fourteen_days.query('group == "A"')['user_id'].nunique()
B = fourteen_days.query('group == "B"')['user_id'].nunique()

count_events_on_users_and_events_group_A = count_events_on_users_and_events_group[count_events_on_users_and_events_group['group'] == 'A']
count_events_on_users_and_events_group_B = count_events_on_users_and_events_group[count_events_on_users_and_events_group['group'] == 'B']

count_events_on_users_and_events_group_A['count_users'] = round(count_events_on_users_and_events_group_A['events_count'] / A,2)
count_events_on_users_and_events_group_B['count_users'] = round(count_events_on_users_and_events_group_B['events_count'] / B,2)

print('Среднее количество событий на пользователя в группе А')
display(count_events_on_users_and_events_group_A[['event_name','count_users']])
print('Среднее количество событий на пользователя в группе В')
display(count_events_on_users_and_events_group_B[['event_name','count_users']])

Среднее количество событий на пользователя в группе А


/tmp/ipykernel_48/1746504860.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_48/1746504860.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,event_name,count_users
0,login,2.16
1,product_cart,0.65
2,product_page,1.38
3,purchase,0.66


Среднее количество событий на пользователя в группе В


,event_name,count_users
4,login,2.22
5,product_cart,0.61
6,product_page,1.21
7,purchase,0.58


In [62]:
#Построим наглядный график
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Группа А", "Группа В"))

fig.add_trace(go.Box(x=count_events_on_users_and_events_group_A['event_name'], y=count_events_on_users_and_events_group_A['count_users']),
              row=1, col=1)

fig.add_trace(go.Box(x=count_events_on_users_and_events_group_B['event_name'], y=count_events_on_users_and_events_group_B['count_users']),
              row=1, col=2)

fig.update_yaxes(title_text="Количество человек", row=1, col=1)
fig.update_yaxes(title_text="Количество человек", row=1, col=2)

**Вывод:**
- Событие `Регистрация` между группами различается: 2.16(A) и 2.22(B) - Группа В лидирует.
- В событиях `product_cart`, `product_page`, `purchase` лидирует группа А.

### Как число событий в выборках распределено по дням?

#### посмотрим в разрезе дней недели

In [63]:
#Добавим столбец с днями недели
fourteen_days['day'] = pd.to_datetime(fourteen_days['first_date']).dt.day_name()

#Сделаем групппировку по выборкам
a_day = fourteen_days.query('group == "A"')
b_day = fourteen_days.query('group == "B"')

a_f_d = a_day.groupby(['day','event_name']).agg(count_event = ('event_name','count')).reset_index()
b_f_d = b_day.groupby(['day','event_name']).agg(count_event = ('event_name','count')).reset_index()

In [64]:
#Построим графики
fig = px.bar(a_f_d, x="count_event", y="event_name", color='day', orientation='h',
             height=400,
             title='Группа А')
fig.update_layout(xaxis_title='Количество событий',
                   yaxis_title='События')

fig_2 = px.bar(b_f_d, x="count_event", y="event_name", color='day', orientation='h',
             height=400,
             title='Группа В')
fig_2.update_layout(xaxis_title='Количество событий',
                   yaxis_title='События')


fig_2.show()
fig.show()

#### посмотрим в разрезе распределения по общему количеству дней

In [65]:
#Построим таблицу
count_day = fourteen_days.groupby(['first_date', 'group']).agg(count = ('event_name','count')).reset_index()

#Построим график
fig = px.bar(count_day, x="first_date", y="count", color = 'group', title='Распределение событий по дням в двух группах')
fig.update_layout(xaxis_title='Дата',
                   yaxis_title='Количество событий')
fig.show()

**Вывод:**

Посмотрим в разрезе дней недели.

- В понедельник 
    - Группа А 
        - топ событие `Регистрация` - 1953
    - Группа В
        - топ событие `Регистрация`, - 782. 
- Вторник
    - Группа А 
        - топ событие `Регистрация` - 827
    - Группа В
        - топ событие `Регистрация`, - 191
- Среда
    - Группа А 
        - топ событие `Регистрация` - 467
    - Группа В
        - топ событие `Регистрация`, - 655
- Четверг
    - Группа А 
        - топ событие `Регистрация` - 636
    - Группа В
        - топ событие `Регистрация`, - 155
- Пятница
    - Группа А 
        - топ событие `Регистрация` - 732
    - Группа В
        - топ событие `Регистрация`, - 133
- Суббота
    - Группа А 
        - топ событие `Регистрация` - 532
    - Группа В
        - топ событие `Регистрация`, - 115
- Воскресенье
    - Группа А 
        - топ событие `Регистрация` - 582
    - Группа В
        - топ событие `Регистрация`, - 217    
        
Посмотрим по общему количеству дней.

Больше событий у группы А прошло 14 декабря. А у группы В 7 декабря.

### Как меняется конверсия в воронке в выборках на разных этапах?

In [66]:
#СОздадим таблицы, где отражены агрегировнные данные по событиям
hopper_a = fourteen_days.query('group == "A"').groupby('event_name')\
.agg(count_events = ('event_name','count'), user_count = ('user_id','nunique')).sort_values(by = 'count_events', ascending = False)
hopper_b = fourteen_days.query('group == "B"').groupby('event_name')\
.agg(count_events = ('event_name','count'), user_count = ('user_id','nunique')).sort_values(by = 'count_events', ascending = False)

In [67]:
#Построим график
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A', y = hopper_a.index, x = hopper_a['user_count'],
    opacity = 0.85, textinfo = "value+percent initial"))

fig.add_trace(go.Funnel(
    name = 'B', y = hopper_b.index, x = hopper_b['user_count'],
    opacity = 0.85, textinfo = "value+percent initial"))

fig.show() 

Воронка конверсии определенно показывает нам, что первоочередным событием является регистарция, потом идет просмотр страницы продукта, покупка и просмотр продуктовой карточки. И только 30% в группе А и 28% в группе В от всех зарегистрированных пользователей совершают покупку. Больше всего падает конверсия из просмотра карточки в покупку 51% у группы А и 52% в группе В. 10% роста конверсий метрик не обнаружено.

### Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

1. Тест попал под маркетинговую компанию `Christmas&New Year Promo EU` и `CIS New Year Gift Lottery` 
2. Было запущено два теста
3. Группа А имеет внушительный перевес 

## Проведите оценку результатов A/B-тестирования

### Что можно сказать про результаты A/B-тестирования?

In [68]:
#создадим таблицу
convers = fourteen_days.groupby(['lifetime', 'group', 'event_name'], as_index=False)['user_id'].count().rename(columns={'user_id':'events_count'})

#таблица с событием login
login = convers.query('event_name == "login"')
#добавим расчет cumsum от количества событий
login['events_cumsum'] = login.groupby('group')['events_count'].cumsum()

# расчёт роста количества событий login для двух групп за 14 дней лайфтайма
a_log = round((login.iloc[-2]['events_cumsum'] - login.iloc[0]['events_cumsum']) 
                        / login.iloc[0]['events_cumsum'], 3) 

b_log = round((login.iloc[-1]['events_cumsum'] - login.iloc[1]['events_cumsum']) 
                        / login.iloc[1]['events_cumsum'], 3) 



#таблица с событием product_page
product_pages = convers.query('event_name == "product_page"')
#добавляем расчёт cumsum от количества событий
product_pages['events_cumsum'] = product_pages.groupby('group')['events_count'].cumsum()

# расчёт роста количества событий product_pages для двух групп за 14 дней лайфтайма
a_prod_page = round((product_pages.iloc[-2]['events_cumsum'] - product_pages.iloc[0]['events_cumsum']) 
                        / product_pages.iloc[0]['events_cumsum'], 3) 

b_prod_page = round((product_pages.iloc[-1]['events_cumsum'] - product_pages.iloc[1]['events_cumsum']) 
                        / product_pages.iloc[1]['events_cumsum'], 3) 



#таблица с событием product_cart
product_carts = convers.query('event_name == "product_cart"')
#добавляем расчёт cumsum от количества событий
product_carts['events_cumsum'] = product_carts.groupby('group')['events_count'].cumsum()

# расчёт роста количества событий product_pages для двух групп за 14 дней лайфтайма
a_product_cart = round((product_carts.iloc[-2]['events_cumsum'] - product_carts.iloc[0]['events_cumsum']) 
                        / product_carts.iloc[0]['events_cumsum'], 3) 

b_product_cart = round((product_carts.iloc[-1]['events_cumsum'] - product_carts.iloc[1]['events_cumsum']) 
                        / product_carts.iloc[1]['events_cumsum'], 3) 



#таблица с событием purchases
purchases_  = convers.query('event_name == "purchase"')
#добавляем расчёт cumsum от количества событий
purchases_['events_cumsum'] = purchases_.groupby('group')['events_count'].cumsum()

# расчёт роста количества событий product_pages для двух групп за 14 дней лайфтайма
a_purchases = round((purchases_.iloc[-2]['events_cumsum'] - purchases_.iloc[0]['events_cumsum']) 
                        / purchases_.iloc[0]['events_cumsum'], 3) 

b_purchases = round((purchases_.iloc[-1]['events_cumsum'] - purchases_.iloc[1]['events_cumsum']) 
                        / purchases_.iloc[1]['events_cumsum'], 3) 


print("Группа А - рост количества событий - регистрация:", round((a_log - 1)*100,2),'%')
print("Группа В - рост количества событий - регистрация:", round((b_log - 1)*100,2),'%')
print("")
print("Группа А - рост количества событий - просмотр страницы товара:", round((a_prod_page - 1)*100,2),'%')
print("Группа В - рост количества событий - просмотр страницы товара:", round((b_prod_page - 1)*100,2),'%')
print("")
print("Группа А - рост количества событий - просмотр карточки продукта:", round((a_product_cart - 1)*100,2),'%')
print("Группа В - рост количества событий - просмотр карточки продукта:", round((b_product_cart - 1)*100,2),'%')
print("")
print("Группа А - рост количества событий - покупка товара:", round((a_purchases - 1)*100,2),'%')
print("Группа В - рост количества событий - покупка товара:", round((b_purchases - 1)*100,2),'%')


Группа А - рост количества событий - регистрация: 16.3 %
Группа В - рост количества событий - регистрация: 26.1 %

Группа А - рост количества событий - просмотр страницы товара: 12.8 %
Группа В - рост количества событий - просмотр страницы товара: 20.0 %

Группа А - рост количества событий - просмотр карточки продукта: 16.0 %
Группа В - рост количества событий - просмотр карточки продукта: 26.8 %

Группа А - рост количества событий - покупка товара: 8.4 %
Группа В - рост количества событий - покупка товара: 12.3 %


/tmp/ipykernel_48/2169365368.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_48/2169365368.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_48/2169365368.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_48/21693

In [77]:
# Автоматизируем процесс
def think_rev(event_name):
    
    funnel = convers.query('event_name == @event_name')
    #добавим расчет cumsum от количества событий
    funnel['events_cumsum'] = funnel.groupby('group')['events_count'].cumsum()

    # расчёт роста количества событий login для двух групп за 14 дней лайфтайма
    a_log = round((funnel.iloc[-2]['events_cumsum'] - funnel.iloc[0]['events_cumsum']) 
                            / funnel.iloc[0]['events_cumsum'], 3) 

    b_log = round((funnel.iloc[-1]['events_cumsum'] - funnel.iloc[1]['events_cumsum']) 
                            / funnel.iloc[1]['events_cumsum'], 3) 
    
    print(f"Группа А - рост количества событий - {event_name}:", round((a_log - 1)*100,2),'%')
    print(f"Группа В - рост количества событий - {event_name}:", round((b_log - 1)*100,2),'%')
    print("")   


think_rev('login')
think_rev('product_page')
think_rev('product_cart')
think_rev('purchase')

Группа А - рост количества событий - login: 16.3 %
Группа В - рост количества событий - login: 26.1 %

Группа А - рост количества событий - product_page: 12.8 %
Группа В - рост количества событий - product_page: 20.0 %

Группа А - рост количества событий - product_cart: 16.0 %
Группа В - рост количества событий - product_cart: 26.8 %

Группа А - рост количества событий - purchase: 8.4 %
Группа В - рост количества событий - purchase: 12.3 %



/tmp/ipykernel_48/3180822016.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Вывод:**

Рост количества событий `Регистрация`
- Группа А - 16.3 %
- Группа В - 26.1 %

Рост количества событий `Просмотр страницы товара`
- Группа А - 12.8 %
- Группа В - 20.0 %

Рост количества событий `Просмотр карточки продукта`
- Группа А - 16.0 %
- Группа В - 26.8 %

Рост количества событий `Покупка товара`
- Группа А - 8.4 %
- Группа В - 12.3 %

### Проверьте статистическую разницу долей z-критерием.

На данном этапе мы проведем сравнение пропорций для следующих метрик:
    `login`,
     `product_page` ,
     `product_cart`,
     `purchase`.

**Нулевая гипотеза**: Доли значений метрик в группах равны.

**Альтернативная гипотеза**: Доли значений метрик в группах не равны.

**Альфа**: Обозначим за 0.5, а метрики у нас 4, 0.5/4 = 0,13

In [78]:
#построим таблицу liftime, group и количество событий
count_users = (fourteen_days.groupby(['lifetime','group'], as_index=False)['user_id'].nunique()
               .rename(columns={'user_id':'users_count'}))


In [79]:
#события типа login
login = login.merge(count_users, on=['lifetime', 'group'])
login['event_by_user'] = login['events_count'] / login['users_count']

#события типа product_page
product_pages = product_pages.merge(count_users, on=['lifetime', 'group'])
product_pages['event_by_user'] = product_pages['events_count'] / product_pages['users_count']

#события типа product_cart
product_carts = product_carts.merge(count_users, on=['lifetime', 'group'])
product_carts['event_by_user'] = product_carts['events_count'] / product_carts['users_count']

#события типа purchase
purchases_ = purchases_.merge(count_users, on=['lifetime', 'group'])
purchases_['event_by_user'] = purchases_['events_count'] / purchases_['users_count']

In [80]:
# смотрим на средние значения ключевых метрик
print('Тип события login')
display(login.groupby('group')['event_by_user'].mean())
print('#'*50)
print('')
print('Тип события product_pages')
display(product_pages.groupby('group')['event_by_user'].mean())
print('#'*50)
print('')
print('Тип события product_carts')
display(product_carts.groupby('group')['event_by_user'].mean())
print('#'*50)
print('')
print('Тип события purchases')
display(purchases_.groupby('group')['event_by_user'].mean())

Тип события login


group
A    0.999936
B    0.999172
Name: event_by_user, dtype: float64

##################################################

Тип события product_pages


group
A    0.632933
B    0.538362
Name: event_by_user, dtype: float64

##################################################

Тип события product_carts


group
A    0.274869
B    0.291066
Name: event_by_user, dtype: float64

##################################################

Тип события purchases


group
A    0.272584
B    0.263668
Name: event_by_user, dtype: float64

Наибольшее отличие в количестве событий имеет метрика product_carts.

In [81]:
#приступим к расчет z - теста
def test_z(df):
    z = ztest(df.query('group == "A"')['event_by_user'], df.query('group == "B"')['event_by_user'], value=0)[1]
    print("p-value:", z)

In [82]:
#Стат. разница между выборками типов login
test_z(login)

p-value: 0.2351186977094617


In [83]:
#Стат. разница между выборками типов product_pages
test_z(product_pages)

p-value: 0.00044194877853921796


In [84]:
#Стат. разница между выборками типов product_carts
test_z(product_carts)

p-value: 0.45802870383641603


In [85]:
#Стат. разница между выборками типов product_carts
test_z(purchases_)

p-value: 0.7305785686680758


**Вывод**

Из 4 рассматриваемых метрик, только одна метрика `product_pages` оказалась меньше альфы, доли этой метрики имеют большую разницу, в отличии от остальных.

## Общий вывод

Вывод будет описан по каждому модулю проекта, всего их 6:
1. Вводная часть: указали цель исследования, ТЗ и описание данных

2. Исследование данных/Предобработка
    * подгрузили 4 датассета
    * преобразовали типы данных, вчастности заменили все типы столбцов с датой
    * посмотрели на пропуски - были только в столбце details 377577 шт, природа их наличия ясна (наличие заполненых строк в details обосновано тем, что там помечается сумма покупки в долларах)

3. Оценка корректности проведения теста
    * Проверили данные требований ТЗ на соответствие (узнали, что проводилось два теста: `recommender_system_test` и `interface_eu_test`)
    * Количество пользователей принимавших участие в двух тестах 1602
    * Дата запуска 2020-12-07
    * Дата остновки набора 2020-12-21
    * Аудитория теста у новых пользователей с Европы равна 15%, что соответствует ТЗ
    * Ожидаемое количество участников теста: 6000. У нас количество участиков 6701.
    * В временной интервал проведения теста попали две акции Christmas&New Year Promo и CIS New Year Gift Lottery. Но первая акция окажет большее влиияние, нежели чем вторая, потому что данная акция проводилась именно в Европе.
    * пользователей, которые входят в группу А и В теста  recommender_system_test не обнаружено
    * пользователи в группах распеределены не равномерно А 3824 пользователя - 57% от общего количества, а в группе В 2877 - 43%

4. Исследовательский анализ данных
    * количество событий на пользователя не совсем одинаково распределены в выборках
        * Событие Регистрация между группами различается: 2.16(A) и 2.22(B) - Группа В лидирует.
        * В событиях product_cart, product_page, purchase лидирует группа А.
    * события по дням недели распределены весьма хаотично, например в понедельник выполняется событий больше , чем в остальные дни недели
    * что касается распределения событий по общему количеству днейЮ который имеется в датассете , большая часть событий была 14 декабря у группы А и 7 декабря у группы В.
    * Воронка конверсии определенно показывает нам, что первоочередным событием является регистарция, потом идет просмотр страницы продукта, покупка и просмотр продуктовой карточки. И только 30% в группе А и 28% в группе В от всех зарегистрированных пользователей совершают покупку. Больше всего падает конверсия из просмотра карточки в покупку 51% у группы А и 52% в группе В.

5. Оценка результатов A/B-тестирования
    * Рост количества событий в целом преобладает у группы В
        * Рост количества событий Регистрация
            * Группа А - 16.3 %
            * Группа В - 26.1 %
        * Рост количества событий Просмотр страницы товара
            * Группа А - 12.8 %
            * Группа В - 20.0 %
        * Рост количества событий Просмотр карточки продукта
            * Группа А - 16.0 %
            * Группа В - 26.8 %
        * Рост количества событий Покупка товара
            * Группа А - 8.4 %
            * Группа В - 12.3 %

При проверке статистической разницы долей z-критерием, мы определили из 4 рассматриваемых метрик, только одна метрика product_pages оказалась меньше альфы, доли этой метрики имеют большую разницу, в отличии от остальных, в целом по результатам А/В теста лучшей из двух групп по показателям 4 метрик - аявляется группа А, однако сам тест не является корректным, пожтому и результаты не совсем однозначны. Например в группе А было больше пользователей чем в группе В, количество участников было более 6000, как ожидалось, активность проявляло в разы меньше.